<a href="https://colab.research.google.com/github/Wilsonmfreire/Treinando_Spark_com_Pyspark/blob/main/Spark_Udemy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Principais configurações
script para rodar pyspark


In [2]:
#Instalando o Java
!apt-get install openjdk-8-jdk-headless -qq >/dev/null

In [3]:
#Baixar a versão mais recente do spark
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

In [4]:
#Dezipar spark
!tar xf /content/spark-3.2.0-bin-hadoop3.2.tgz

In [5]:
#Criar variável de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2/"

In [6]:
#instalar a lib findspark que ajuda a localizar o Spark no sistema e importá-lo como uma biblioteca regulgar
!pip install -q findspark

In [7]:
#import a lib findspark
import findspark
findspark.init()

In [124]:
#criar spark session

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import  *
from pyspark.sql.functions import  col, lit, coalesce
from pyspark.sql import functions as Func

spark = SparkSession.builder\
        .master('local')\
        .appName('sparkcolab')\
        .getOrCreate()

In [9]:
#Upload de arquivos
#from google.colab import files
#arquivo = files.upload()

In [10]:
!wget www.datascientist.com.br/bigdata/download.zip

--2021-12-15 10:28:52--  http://www.datascientist.com.br/bigdata/download.zip
Resolving www.datascientist.com.br (www.datascientist.com.br)... 108.179.192.203
Connecting to www.datascientist.com.br (www.datascientist.com.br)|108.179.192.203|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207452 (203K) [application/zip]
Saving to: ‘download.zip’

download.zip        100%[===================>] 202.59K   292KB/s    in 0.7s    

2021-12-15 10:28:54 (292 KB/s) - ‘download.zip’ saved [207452/207452]



In [11]:
!unzip /content/download.zip

Archive:  /content/download.zip
   creating: download/Atividades/
  inflating: download/Atividades/Clientes.parquet  
  inflating: download/Atividades/ItensVendas.parquet  
  inflating: download/Atividades/Produtos.parquet  
  inflating: download/Atividades/Vendas.parquet  
  inflating: download/Atividades/Vendedores.parquet  
  inflating: download/Carros.csv     
  inflating: download/Churn.csv      
   creating: download/demo/
  inflating: download/demo/1.CreateTable.sql  
  inflating: download/demo/2.InsertClientes.sql  
  inflating: download/demo/3.InsertIntoProdutos.sql  
  inflating: download/demo/4.InsertIntoVendedores.sql  
  inflating: download/demo/5.InsertIntoVendas.sql  
  inflating: download/demo/6.InsertItensVenda.sql  
  inflating: download/despachantes.csv  
  inflating: download/iris.csv       
   creating: download/mongo/
  inflating: download/mongo/posts.json  
  inflating: download/reclamacoes.csv  
   creating: download/streaming/
  inflating: download/streaming/1.

##Principais transformações
DataFrames e RDDS

In [12]:
#Criando primeiro dataFrame
df1 = spark.createDataFrame([("Pedro", 10),("Maria", 20),("Jose",40)])
df1.show( )

+-----+---+
|   _1| _2|
+-----+---+
|Pedro| 10|
|Maria| 20|
| Jose| 40|
+-----+---+



In [13]:
#criando dataFrame com StructField
#nao entendi pq passar o data no createDaaFrame deu certo
#dados = [("1", "Pedro"), 
#      ("2", "Maria")
#    ]
#
#schema = StructType([ \
#    StructField("id", StringType(), True),\
#    StructField("nome", StringType(), True),\
#  ])

#df2 = spark.createDataFrame(data=data2,schema=schema)
#df2.printSchema()
#df2.show()

In [14]:
#criando dataFrame sem StructField
schema2 = "Produtos STRING, Vendas INT"
vendas = [["Caneta", 10], ["Lápis", 20], ["Caneta", 40]]
df3 = spark.createDataFrame(vendas, schema2)
df3.show()

+--------+------+
|Produtos|Vendas|
+--------+------+
|  Caneta|    10|
|   Lápis|    20|
|  Caneta|    40|
+--------+------+



In [15]:
#trabalhando com sum
#precisamos agrupar o que não iremos somar e usar .agg(sum("passar coluna com valores"))
agrupados = df3.groupBy("Produtos").agg(sum("vendas"))
agrupados.show()

+--------+-----------+
|Produtos|sum(vendas)|
+--------+-----------+
|  Caneta|         50|
|   Lápis|         20|
+--------+-----------+



In [16]:
#passando o dataFrame direto sem associar ao um novo
df3.groupBy("Produtos").agg(sum("Vendas")).show()

+--------+-----------+
|Produtos|sum(Vendas)|
+--------+-----------+
|  Caneta|         50|
|   Lápis|         20|
+--------+-----------+



In [17]:
# Trabalhando com select e expressões
df3.select('Vendas').show()
df3.select('Produtos',\
           'Vendas',\
           expr('Vendas *0.2')).show()

+------+
|Vendas|
+------+
|    10|
|    20|
|    40|
+------+

+--------+------+--------------+
|Produtos|Vendas|(Vendas * 0.2)|
+--------+------+--------------+
|  Caneta|    10|           2.0|
|   Lápis|    20|           4.0|
|  Caneta|    40|           8.0|
+--------+------+--------------+



In [18]:
#Trabalhando com exp e select
df3.select("Vendas").show()
df3.select("Produtos", "Vendas", expr("Vendas *0.2")).show()

+------+
|Vendas|
+------+
|    10|
|    20|
|    40|
+------+

+--------+------+--------------+
|Produtos|Vendas|(Vendas * 0.2)|
+--------+------+--------------+
|  Caneta|    10|           2.0|
|   Lápis|    20|           4.0|
|  Caneta|    40|           8.0|
+--------+------+--------------+



In [19]:
#Atribuindo um schema
#Ler arquivo csv
arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
despachantes = spark.read.csv(
    "/content/download/despachantes.csv",\
    header=False,\
    schema= arqschema\
    )
despachantes.show()
despachantes.printSchema()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+

root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- status: 

In [20]:
#Auto schema do spark
#Ler arquivo csv
#Arquivo sem nome de colunas
desp_autoSchema = spark.read.load(
    "/content/download/despachantes.csv",\
    header = False,\
    format = "csv",\
    sep = ",",\
    inferSchema = True
)
desp_autoSchema.show()
desp_autoSchema.printSchema()

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

In [21]:
#Trabalhando com operadores
#<, >, &, |
despachantes.select('id',\
                    'nome',\
                    'vendas')\
                    .where(Func.col('vendas')>20)\
                    .show()

despachantes.select('id',\
                    'nome',\
                    'vendas')\
                    .where((Func.col('vendas')>20) & (Func.col('vendas')<40))\
                    .show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
+---+-------------------+------+

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
+---+-------------------+------+



In [22]:
despachantes.select\
              ('id', 'nome','vendas')\
            .where(Func.col('vendas')>20).show()
despachantes.select\
              ('id', 'nome','vendas')\
            .where((Func.col('vendas')>20) & (Func.col('vendas')<40)).show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
+---+-------------------+------+

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
+---+-------------------+------+



In [23]:
# Alterando nome das colunas do dataFrame 
# Vamos usar o comando withColumnRenamed('colunaAtua','novoNome')
# Atencao que printando o schema ele tambem sofre alteracao
despachantes2 = despachantes.withColumnRenamed('_c0', 'id')
despachantes2 = despachantes2.withColumnRenamed('_c1', 'nome')
despachantes2 = despachantes2.withColumnRenamed('_c2', 'status')
despachantes2 = despachantes2.withColumnRenamed('_c3', 'cidade')
despachantes2 = despachantes2.withColumnRenamed('_c4', 'vendas')
despachantes2 = despachantes2.withColumnRenamed('_c5', 'data')

despachantes2.columns
despachantes2.show()
despachantes2.printSchema()


+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+

root
 |-- id: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- status: 

In [24]:
# Alterando tipo string para date com o comando .withColumn passando como paramentro to_timestamp(Fun.col('coluna'), 'yyyy-MM-dd')
# convertendo para date  
despachantes2 = despachantes2.withColumn('data', to_timestamp(Func.col('data'), 'yyyy-MM-dd'))
despachantes2.schema  

StructType(List(StructField(id,IntegerType,true),StructField(nome,StringType,true),StructField(status,StringType,true),StructField(cidade,StringType,true),StructField(vendas,IntegerType,true),StructField(data,TimestampType,true)))

In [25]:
# Praticando distinct
despachantes2.select(year('data')).distinct().show()

+----------+
|year(data)|
+----------+
|      2018|
|      2019|
|      2020|
+----------+



In [26]:
# Praticando orderBy
despachantes2.select('nome',year('data')).orderBy('nome').show()

+-------------------+----------+
|               nome|year(data)|
+-------------------+----------+
|   Carminda Pestana|      2020|
|    Deolinda Vilela|      2020|
|   Emídio Dornelles|      2020|
|Felisbela Dornelles|      2020|
|     Graça Ornellas|      2020|
|   Matilde Rebouças|      2019|
|    Noêmia   Orriça|      2019|
|      Roque Vásquez|      2020|
|      Uriel Queiroz|      2018|
|   Viviana Sequeira|      2020|
+-------------------+----------+



In [27]:
# Praticando groupBy com count
despachantes2.select('data').groupBy(year('data')).count().show()

+----------+-----+
|year(data)|count|
+----------+-----+
|      2018|    1|
|      2019|    2|
|      2020|    7|
+----------+-----+



In [28]:
# Praticando sum
despachantes2.select(Func.sum('vendas')).show()

+-----------+
|sum(vendas)|
+-----------+
|        325|
+-----------+



In [29]:
# Praticando orderBy
despachantes2.orderBy(Func.col('vendas').desc()).show()

+---+-------------------+------+-------------+------+-------------------+
| id|               nome|status|       cidade|vendas|               data|
+---+-------------------+------+-------------+------+-------------------+
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05 00:00:00|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05 00:00:00|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05 00:00:00|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05 00:00:00|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05 00:00:00|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05 00:00:00|
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11 00:00:00|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05 00:00:00|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05 00:00:00|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05 00:00:00|
+---+-------------------+------+------

In [30]:
# Praticando orderBy 
# ordenando nome asc
# ordenando cidade desc
# ordenando vendas desc
despachantes2.orderBy(Func.col('nome').asc(),\
                      Func.col('cidade').desc(),\
                      Func.col('vendas').desc()).show()

+---+-------------------+------+-------------+------+-------------------+
| id|               nome|status|       cidade|vendas|               data|
+---+-------------------+------+-------------+------+-------------------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11 00:00:00|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05 00:00:00|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05 00:00:00|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05 00:00:00|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05 00:00:00|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05 00:00:00|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05 00:00:00|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05 00:00:00|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05 00:00:00|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05 00:00:00|
+---+-------------------+------+------

In [31]:
# Praticando groupBy com sum 
# somando quantidade de vendas por cidade
despachantes2.groupBy('cidade')\
              .agg(sum('vendas')).show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
|  Santa Maria|         68|
|Novo Hamburgo|         34|
| Porto Alegre|        223|
+-------------+-----------+



In [32]:
# Praticando groupBy, sum, orderby
# agrupando cidade
# somando vendas
# ordenando coluna do resultado sum
despachantes2.groupBy('cidade')\
             .agg(sum('vendas'))\
             .orderBy(Func.col('sum(vendas)'))\
             .show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
|Novo Hamburgo|         34|
|  Santa Maria|         68|
| Porto Alegre|        223|
+-------------+-----------+



In [33]:
# Praticando filter
# buscando nome especifico da tabela
despachantes2.filter(Func.col('nome') == 'Deolinda Vilela').show()

+---+---------------+------+-------------+------+-------------------+
| id|           nome|status|       cidade|vendas|               data|
+---+---------------+------+-------------+------+-------------------+
|  2|Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05 00:00:00|
+---+---------------+------+-------------+------+-------------------+



##Manipulando arquivos


>Alterando nome de arquivos\
>Salvando dataFrames em formatos diferentes\
>Importando arquivos

In [34]:
# Salvando dataFrame despachantes2 
# formato parquet, csv, json, orc
despachantes2.write.format('parquet').save('/content/dfimportparquet')

In [35]:
despachantes2.write.format('csv').save('/content/dfimport_csv')

In [36]:
despachantes2.write.format('json').save('/content/dfimport_json')

In [37]:
despachantes2.write.format('orc').save('/content//dfimport_orc')

In [61]:
mv part-00000-38022888-0f11-431f-bdd2-e123838450d8-c000.snappy.parquet despachantes.parquet

In [62]:
par = spark.read.format('parquet').load('/content/dfimportparquet/despachantes.parquet')
par.show()

+---+-------------------+------+-------------+------+-------------------+
| id|               nome|status|       cidade|vendas|               data|
+---+-------------------+------+-------------+------+-------------------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11 00:00:00|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05 00:00:00|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05 00:00:00|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05 00:00:00|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05 00:00:00|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05 00:00:00|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05 00:00:00|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05 00:00:00|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05 00:00:00|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05 00:00:00|
+---+-------------------+------+------

In [64]:
ls

despachantes.parquet  _SUCCESS


In [65]:
cd ..

/content


In [67]:
cd dfimport_csv/

/content/dfimport_csv


In [69]:
mv part-00000-e8f62f28-47c9-4034-b39d-0a1d9d7349e6-c000.csv despachantes.csv

In [70]:
cd ..

/content


In [71]:
cd dfimport_json/

/content/dfimport_json


In [73]:
mv part-00000-60a35045-642e-4eaf-a9a7-99c2d1f5e2ce-c000.json despachantes.json

In [74]:
cd ..

/content


In [76]:
cd dfimport_orc/

/content/dfimport_orc


In [77]:
mv part-00000-221909b8-7824-4358-bec4-975fdfab31a9-c000.snappy.orc despachantes.orc

In [79]:
cd ..

/content


In [78]:
js = spark.read.format('json').load('/content/dfimport_json/despachantes.json')
js.show()
js.printSchema()

+-------------+--------------------+---+-------------------+------+------+
|       cidade|                data| id|               nome|status|vendas|
+-------------+--------------------+---+-------------------+------+------+
|  Santa Maria|2020-08-11T00:00:...|  1|   Carminda Pestana| Ativo|    23|
|Novo Hamburgo|2020-03-05T00:00:...|  2|    Deolinda Vilela| Ativo|    34|
| Porto Alegre|2020-02-05T00:00:...|  3|   Emídio Dornelles| Ativo|    34|
| Porto Alegre|2020-02-05T00:00:...|  4|Felisbela Dornelles| Ativo|    36|
| Porto Alegre|2020-02-05T00:00:...|  5|     Graça Ornellas| Ativo|    12|
| Porto Alegre|2019-01-05T00:00:...|  6|   Matilde Rebouças| Ativo|    22|
|  Santa Maria|2019-10-05T00:00:...|  7|    Noêmia   Orriça| Ativo|    45|
| Porto Alegre|2020-03-05T00:00:...|  8|      Roque Vásquez| Ativo|    65|
| Porto Alegre|2018-05-05T00:00:...|  9|      Uriel Queiroz| Ativo|    54|
| Porto Alegre|2020-09-05T00:00:...| 10|   Viviana Sequeira| Ativo|     0|
+-------------+----------

In [80]:
csv = spark.read.format('csv').load('/content/dfimport_csv/despachantes.csv')
csv.show()
csv.printSchema()

+---+-------------------+-----+-------------+---+--------------------+
|_c0|                _c1|  _c2|          _c3|_c4|                 _c5|
+---+-------------------+-----+-------------+---+--------------------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11T00:00:...|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05T00:00:...|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05T00:00:...|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05T00:00:...|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05T00:00:...|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05T00:00:...|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05T00:00:...|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05T00:00:...|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05T00:00:...|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05T00:00:...|
+---+-------------------+-----+-------------+---+--------------------+

root


In [81]:
orc = spark.read.format('orc').load('/content/dfimport_orc/despachantes.orc')
orc.show()
orc.printSchema()

+---+-------------------+------+-------------+------+-------------------+
| id|               nome|status|       cidade|vendas|               data|
+---+-------------------+------+-------------+------+-------------------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11 00:00:00|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05 00:00:00|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05 00:00:00|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05 00:00:00|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05 00:00:00|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05 00:00:00|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05 00:00:00|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05 00:00:00|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05 00:00:00|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05 00:00:00|
+---+-------------------+------+------

In [82]:
cs2 = spark.read.format('csv').load('/content/dfimport_csv/despachantes.csv', schema = arqschema)
cs2.show()
cs2.printSchema()

+---+-------------------+------+-------------+------+--------------------+
| id|               nome|status|       cidade|vendas|                data|
+---+-------------------+------+-------------+------+--------------------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11T00:00:...|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05T00:00:...|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05T00:00:...|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05T00:00:...|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05T00:00:...|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05T00:00:...|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05T00:00:...|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05T00:00:...|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05T00:00:...|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05T00:00:...|
+---+-------------------+

# Faça você mesmo 

No 'Manipulando arquivos' treinei navegação entre pastas, agora irei navegar dentro dos (  ) até chegar no destino desejado!

In [94]:
# importando arquivo clientes no formado parquet já com schema
clientes = spark.read.load('/content/download/Atividades/Clientes.parquet')
clientes.show(2)
clientes.printSchema()

+---------+--------------------+------+------+------+
|ClienteID|             Cliente|Estado|Genero|Status|
+---------+--------------------+------+------+------+
|        1|Adelina Buenaventura|    RJ|     M|Silver|
|        2|        Adelino Gago|    RJ|     M|Silver|
+---------+--------------------+------+------+------+
only showing top 2 rows

root
 |-- ClienteID: long (nullable = true)
 |-- Cliente: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Status: string (nullable = true)



In [96]:
# importando arquivo vendas no formado parquet já com schema
vendas = spark.read.load('/content/download/Atividades/Vendas.parquet')
vendas.show(2)
vendas.printSchema()

+--------+----------+---------+--------+------+
|VendasID|VendedorID|ClienteID|    Data| Total|
+--------+----------+---------+--------+------+
|       1|         1|       91|1/1/2019|8053.6|
|       2|         6|      185|1/1/2020| 150.4|
+--------+----------+---------+--------+------+
only showing top 2 rows

root
 |-- VendasID: long (nullable = true)
 |-- VendedorID: long (nullable = true)
 |-- ClienteID: long (nullable = true)
 |-- Data: string (nullable = true)
 |-- Total: double (nullable = true)



In [98]:
# importando arquivo vendedores no formado parquet já com schema
vendedores = spark.read.load('/content/download/Atividades/Vendedores.parquet')
vendedores.show(2)
vendedores.printSchema()

+----------+----------------+
|VendedorID|        Vendedor|
+----------+----------------+
|         1|    Armando Lago|
|         2|Capitolino Bahía|
+----------+----------------+
only showing top 2 rows

root
 |-- VendedorID: long (nullable = true)
 |-- Vendedor: string (nullable = true)



In [104]:
atividade1 = clientes.select('Cliente',\
                'Estado',\
                'Status')
atividade1.show()

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



In [130]:
clientes.select('*').where((Func.col('Status') == 'Gold') or (Func.col('Status') == 'Platinum')).show()

ValueError: ignored

In [125]:
clientes.select\
        ('Cliente', 'Status')\
        .where((Func.col('Status') == 'Gold') │ (Func.col('Status') == 'Platinum'))\
        .show()

SyntaxError: ignored